# 1. Variables climáticas

## 1.1 Importación

In [1]:
import pandas as pd
import numpy as np

### Dataset procesado antes de clima

In [2]:
df_procesado = pd.read_csv(r'df_procesado_todas_las_aerolneas.csv')

### Dataset mapeo aeropuertos-estaciones

In [3]:
airport_station_mapping = pd.read_csv('airport_station_mapping.csv')

### Dataset clima desde NOAA

In [4]:
df_output = pd.read_csv(r'Base_climaNOAA\lcd_climate_data.csv')

C:\Users\Karen\AppData\Local\Temp\ipykernel_22444\3311282411.py:1: DtypeWarning: Columns (5,11,16,19,23,27,33,34,35,36,37,39,40,41,42,45,60,61,62,63,64,65,66,71,77,78,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,115,117,118,120,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  df_output = pd.read_csv(r'Base_climaNOAA\lcd_climate_data.csv')


## 1.2 Variables seleccionadas de dataset clima

In [5]:
df_output.head(2)

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,SOURCE,HourlyAltimeterSetting,HourlyDewPointTemperature,...,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,BackupEquipment,BackupLatitude,BackupLongitude,BackupName,WindEquipmentChangeDate
0,14737,2024-10-01T00:00:00,40.64985,-75.44771,118.9,"ALLENTOWN LEHIGH VALLEY INTERNATIONAL AIRPORT,...",SOD,NaN,NaN,NaN,...,NaN,NaN,NaN,"SNOWFALL, SNOWDPTH, SNOWFALLEQ, SNOWWTREQ",NaN,"SNOWBOARD, SNOWSTICK, SRG",40.6254,-75.4403,ALLENTOWN,09/30/2008
1,14737,2024-10-01T00:51:00,40.64985,-75.44771,118.9,"ALLENTOWN LEHIGH VALLEY INTERNATIONAL AIRPORT,...",FM-15,343.0,1017.6,12.8,...,NaN,NaN,NaN,"SNOWFALL, SNOWDPTH, SNOWFALLEQ, SNOWWTREQ",NaN,"SNOWBOARD, SNOWSTICK, SRG",40.6254,-75.4403,ALLENTOWN,09/30/2008


In [6]:
list(df_output.columns)

['STATION',
 'DATE',
 'LATITUDE',
 'LONGITUDE',
 'ELEVATION',
 'NAME',
 'REPORT_TYPE',
 'SOURCE',
 'HourlyAltimeterSetting',
 'HourlyDewPointTemperature',
 'HourlyDryBulbTemperature',
 'HourlyPrecipitation',
 'HourlyPresentWeatherType',
 'HourlyPressureChange',
 'HourlyPressureTendency',
 'HourlyRelativeHumidity',
 'HourlySkyConditions',
 'HourlySeaLevelPressure',
 'HourlyStationPressure',
 'HourlyVisibility',
 'HourlyWetBulbTemperature',
 'HourlyWindDirection',
 'HourlyWindGustSpeed',
 'HourlyWindSpeed',
 'Sunrise',
 'Sunset',
 'DailyAverageDewPointTemperature',
 'DailyAverageDryBulbTemperature',
 'DailyAverageRelativeHumidity',
 'DailyAverageSeaLevelPressure',
 'DailyAverageStationPressure',
 'DailyAverageWetBulbTemperature',
 'DailyAverageWindSpeed',
 'DailyCoolingDegreeDays',
 'DailyDepartureFromNormalAverageTemperature',
 'DailyHeatingDegreeDays',
 'DailyMaximumDryBulbTemperature',
 'DailyMinimumDryBulbTemperature',
 'DailyPeakWindDirection',
 'DailyPeakWindSpeed',
 'DailyPrecipit

In [7]:
var_selecciondas = ['STATION', 'DATE','HourlyDryBulbTemperature', 'DailyAverageDryBulbTemperature', 
                    'HourlyRelativeHumidity', 'DailyAverageRelativeHumidity',
                     'HourlyStationPressure', 'DailyAverageStationPressure',
                     'HourlyVisibility',
                     'HourlyWindSpeed', 'DailyAverageWindSpeed',
                     'HourlyPresentWeatherType', 'DailyWeather']

In [8]:
df_clima_select = df_output[var_selecciondas].copy()

In [9]:
df_clima_select.head(2)

,STATION,DATE,HourlyDryBulbTemperature,DailyAverageDryBulbTemperature,HourlyRelativeHumidity,DailyAverageRelativeHumidity,HourlyStationPressure,DailyAverageStationPressure,HourlyVisibility,HourlyWindSpeed,DailyAverageWindSpeed,HourlyPresentWeatherType,DailyWeather
0,14737,2024-10-01T00:00:00,NaN,17.5,NaN,73.0,NaN,1003.0,NaN,NaN,3.2,NaN,NaN
1,14737,2024-10-01T00:51:00,15.6,NaN,84.0,NaN,1003.6,NaN,16.093,3.1,NaN,NaN,NaN


In [10]:
df_clima_select.isna().sum()

STATION                                 0
DATE                                    0
HourlyDryBulbTemperature           119190
DailyAverageDryBulbTemperature    3752241
HourlyRelativeHumidity             125135
DailyAverageRelativeHumidity      3831673
HourlyStationPressure              452957
DailyAverageStationPressure       3831778
HourlyVisibility                   115952
HourlyWindSpeed                    128958
DailyAverageWindSpeed             3762406
HourlyPresentWeatherType          3075895
DailyWeather                      3802536
dtype: int64

# 2. Imputación de NaN

## 2.1 Llena datos de Hourly con el dato de daily si no existe

In [11]:
pairs = [
    ('HourlyDryBulbTemperature', 'DailyAverageDryBulbTemperature'),
    ('HourlyRelativeHumidity', 'DailyAverageRelativeHumidity'),
    ('HourlyStationPressure', 'DailyAverageStationPressure'),
    ('HourlyWindSpeed', 'DailyAverageWindSpeed'),
    ('HourlyPresentWeatherType', 'DailyWeather')
]

for hourly, daily in pairs:
    df_clima_select[hourly] = df_clima_select[hourly].fillna(df_clima_select[daily])

df_clima_select.head(2)

,STATION,DATE,HourlyDryBulbTemperature,DailyAverageDryBulbTemperature,HourlyRelativeHumidity,DailyAverageRelativeHumidity,HourlyStationPressure,DailyAverageStationPressure,HourlyVisibility,HourlyWindSpeed,DailyAverageWindSpeed,HourlyPresentWeatherType,DailyWeather
0,14737,2024-10-01T00:00:00,17.5,17.5,73.0,73.0,1003.0,1003.0,NaN,3.2,3.2,NaN,NaN
1,14737,2024-10-01T00:51:00,15.6,NaN,84.0,NaN,1003.6,NaN,16.093,3.1,NaN,NaN,NaN


In [12]:
df_clima_select.isna().sum()

STATION                                 0
DATE                                    0
HourlyDryBulbTemperature            20014
DailyAverageDryBulbTemperature    3752241
HourlyRelativeHumidity             105391
DailyAverageRelativeHumidity      3831673
HourlyStationPressure              433318
DailyAverageStationPressure       3831778
HourlyVisibility                   115952
HourlyWindSpeed                     39947
DailyAverageWindSpeed             3762406
HourlyPresentWeatherType          3027014
DailyWeather                      3802536
dtype: int64

## 2.2 Añade horas faltantes por día

In [13]:
# 1. Asegura que DATE sea datetime (para ordenar)
df_clima_select['DATE'] = pd.to_datetime(df_clima_select['DATE'])

# 2. Identifica los días únicos por estación
# Usa .dt.date para obtener solo el día sin la hora específica
dias_por_estacion = df_clima_select[['STATION', 'DATE']].copy()
dias_por_estacion['SOLO_DIA'] = dias_por_estacion['DATE'].dt.date
combinaciones_unicas = dias_por_estacion[['STATION', 'SOLO_DIA']].drop_duplicates()

# 3. Crea el "reloj" de las 24 horas (00:00, 01:00, ..., 23:00)
horas_ideal = pd.DataFrame({'Hour': range(24)})

# 4. Crea el esqueleto combinando Estación + Día + 24 Horas
esqueleto = combinaciones_unicas.merge(horas_ideal, how='cross')

# Convierte el esqueleto a formato datetime para poder comparar/unir
esqueleto['DATE_SKELETON'] = pd.to_datetime(
    esqueleto['SOLO_DIA'].astype(str) + ' ' + esqueleto['Hour'].astype(str) + ':00:00'
)

# 5. Une el esqueleto con los datos originales
# Esto pondrá los datos reales donde coincidan y creará filas nuevas con NaN donde no.
df_expandido = pd.merge(
    df_clima_select, 
    esqueleto[['STATION', 'DATE_SKELETON']], 
    left_on=['STATION', 'DATE'], 
    right_on=['STATION', 'DATE_SKELETON'], 
    how='outer'
)

# 6. Consolida la columna de tiempo
# Si la fecha original era NaN (porque es una hora nueva), usa la del esqueleto
df_expandido['DATE'] = df_expandido['DATE'].fillna(df_expandido['DATE_SKELETON'])
df_expandido.drop(columns=['DATE_SKELETON'], inplace=True)

# 7. ORDENA (orden cronológico)
# Esto pondrá 00:51 antes de 01:00 automáticamente
df_expandido = df_expandido.sort_values(by=['STATION', 'DATE']).reset_index(drop=True)

In [14]:
df_expandido.head(5)

,STATION,DATE,HourlyDryBulbTemperature,DailyAverageDryBulbTemperature,HourlyRelativeHumidity,DailyAverageRelativeHumidity,HourlyStationPressure,DailyAverageStationPressure,HourlyVisibility,HourlyWindSpeed,DailyAverageWindSpeed,HourlyPresentWeatherType,DailyWeather
0,3017,2024-10-01 00:00:00,14.2,14.2,34.0,34.0,840.2,840.2,NaN,3.5,3.5,NaN,NaN
1,3017,2024-10-01 00:53:00,7.2,NaN,44.0,NaN,845.6,NaN,16.093,2.1,NaN,NaN,NaN
2,3017,2024-10-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3017,2024-10-01 01:53:00,6.7,NaN,45.0,NaN,845.3,NaN,16.093,2.6,NaN,NaN,NaN
4,3017,2024-10-01 02:00:00,6.7,NaN,45.0,NaN,847.0,NaN,16.0,2.6,NaN,NaN,NaN


In [15]:
df_expandido.columns

Index(['STATION', 'DATE', 'HourlyDryBulbTemperature',
       'DailyAverageDryBulbTemperature', 'HourlyRelativeHumidity',
       'DailyAverageRelativeHumidity', 'HourlyStationPressure',
       'DailyAverageStationPressure', 'HourlyVisibility', 'HourlyWindSpeed',
       'DailyAverageWindSpeed', 'HourlyPresentWeatherType', 'DailyWeather'],
      dtype='object')

## 2.3 Calcula promedios entre datos

In [16]:
# 1. Define la lista de columnas por interpolar
columnas_hourly = ['HourlyDryBulbTemperature', 'HourlyRelativeHumidity', 
                   'HourlyStationPressure', 'HourlyVisibility', 'HourlyWindSpeed']

for col in columnas_hourly:
    if col in df_expandido.columns:
        # Convierte a string, quita espacios y convierte a numérico
        # errors='coerce' transformará cualquier texto no válido en NaN para poder interpolarlo
        df_expandido[col] = pd.to_numeric(df_expandido[col], errors='coerce')

df_expandido = df_expandido.infer_objects(copy=False)

# 2. Asegura que el DF esté ordenado por estación y fecha antes de interpolar
df_expandido = df_expandido.sort_values(['STATION', 'DATE'])

# 3. Aplica la interpolación lineal por grupo (Estación)
# El método 'linear' busca el valor anterior y posterior y calcula el promedio ponderado
for col in columnas_hourly:
    if col in df_expandido.columns:
        df_expandido[col] = df_expandido.groupby('STATION')[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )

# 4. Redondeo (para que la humedad o visibilidad no tengan demasiados decimales)
df_expandido[columnas_hourly] = df_expandido[columnas_hourly].round(2)

In [17]:
# Rellena texto con el valor anterior más cercano
columnas_texto = ['HourlyPresentWeatherType']
for col in columnas_texto:
    if col in df_expandido.columns:
        df_expandido[col] = df_expandido.groupby('STATION')[col].ffill().bfill()

In [18]:
# 1. Conteo total de NaNs por columna
print("--- Conteo total de NaNs por columna ---")
print(df_expandido[columnas_hourly].isna().sum())

# 2. Ver porcentaje de limpieza
print("\n--- Porcentaje de datos faltantes (%) ---")
print((df_expandido[columnas_hourly].isna().mean() * 100).round(2))

# 3. Identificar estaciones que siguen con muchos nulos
# Esto dirá si hay alguna estación "fantasma" que no tiene ningún dato diario ni horario
estaciones_criticas = df_expandido.groupby('STATION')[columnas_hourly].apply(lambda x: x.isna().sum().sum())
print("\n--- Estaciones con más datos faltantes (Top 5) ---")
print(estaciones_criticas.sort_values(ascending=False).head(5))

--- Conteo total de NaNs por columna ---
HourlyDryBulbTemperature        0
HourlyRelativeHumidity          0
HourlyStationPressure       14716
HourlyVisibility                0
HourlyWindSpeed                 0
dtype: int64

--- Porcentaje de datos faltantes (%) ---
HourlyDryBulbTemperature    0.00
HourlyRelativeHumidity      0.00
HourlyStationPressure       0.25
HourlyVisibility            0.00
HourlyWindSpeed             0.00
dtype: float64

--- Estaciones con más datos faltantes (Top 5) ---
STATION
93843    14716
3017         0
24127        0
24090        0
24089        0
dtype: int64


In [19]:
df_expandido.head(5)

,STATION,DATE,HourlyDryBulbTemperature,DailyAverageDryBulbTemperature,HourlyRelativeHumidity,DailyAverageRelativeHumidity,HourlyStationPressure,DailyAverageStationPressure,HourlyVisibility,HourlyWindSpeed,DailyAverageWindSpeed,HourlyPresentWeatherType,DailyWeather
0,3017,2024-10-01 00:00:00,14.20,14.2,34.0,34.0,840.20,840.2,16.09,3.50,3.5,RA,NaN
1,3017,2024-10-01 00:53:00,7.20,NaN,44.0,NaN,845.60,NaN,16.09,2.10,NaN,RA,NaN
2,3017,2024-10-01 01:00:00,6.95,NaN,44.5,NaN,845.45,NaN,16.09,2.35,NaN,RA,NaN
3,3017,2024-10-01 01:53:00,6.70,NaN,45.0,NaN,845.30,NaN,16.09,2.60,NaN,RA,NaN
4,3017,2024-10-01 02:00:00,6.70,NaN,45.0,NaN,847.00,NaN,16.00,2.60,NaN,RA,NaN


## 2.4 Llena nan de presión (Pressure)

In [20]:
# 1. Crea columna de mes para el análisis
df_expandido['Month'] = df_expandido['DATE'].dt.month

# 2. Cuenta cuántos datos NO NULOS hay de presión por estación y mes
analisis_presion = df_expandido.groupby(['STATION', 'Month'])['HourlyStationPressure'].count().unstack(fill_value=0)

# 3. Ver el resumen de la estación "sospechosa" 93843
print("--- Disponibilidad de Presión para la Estación 93843 por Mes ---")
if '93843' in analisis_presion.index or 93843 in analisis_presion.index:
    print(analisis_presion.loc[93843])
else:
    print("La estación 93843 no aparece en el índice.")

# 4. Ver el resumen general de todas las estaciones (Primeras 10)
print("\n--- Conteo de datos de presión (Primeras 10 estaciones) ---")
print(analisis_presion.head(10))

--- Disponibilidad de Presión para la Estación 93843 por Mes ---
Month
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
Name: 93843, dtype: int64

--- Conteo de datos de presión (Primeras 10 estaciones) ---
Month      1     2     3     4     5     6     7     8     9     10    11  \
STATION                                                                     
3017     1728  1520  1627  1650  1693  1595  1635  1535  1461  1534  1633   
3102     1545  1484  1603  1516  1603  1531  1530  1453  1425  1503  1466   
3145     1445  1344  1497  1441  1487  1419  1477  1442  1381  1435  1412   
3802     1999  1697  1606  1672  1832  1742  1917  1610  1597  1669  1793   
3812     1649  1566  1624  1695  1860  1756  1875  1926  1854  1588  1798   
3820     1594  1616  1682  1664  1834  1648  1706  1785  1594  1645  1804   
3822     1566  1511  1602  1544  1713  1603  1663  1682  1472  1558  1681   
3856     1734  1617  1612  1656  1811  1651  1648  16

In [21]:
# 1. Calcula el promedio de presión de todas las estaciones por cada mes
# Esto crea un "perfil de presión mensual" confiable
map_presion_mensual = df_expandido.groupby('Month')['HourlyStationPressure'].mean()

# 2. Aplica el relleno: Si es NaN, ponemos el promedio del mes correspondiente
# Usa 'map' porque es mucho más rápido que un 'apply' en datasets grandes
df_expandido['HourlyStationPressure'] = df_expandido['HourlyStationPressure'].fillna(
    df_expandido['Month'].map(map_presion_mensual)
)

# 3. Verificación final de nulos en todo el DataFrame
print("--- Conteo Final de NaNs (Misión: 0 nulos) ---")
print(df_expandido[['HourlyDryBulbTemperature', 'HourlyRelativeHumidity', 
                    'HourlyStationPressure', 'HourlyVisibility', 'HourlyWindSpeed']].isna().sum())


--- Conteo Final de NaNs (Misión: 0 nulos) ---
HourlyDryBulbTemperature    0
HourlyRelativeHumidity      0
HourlyStationPressure       0
HourlyVisibility            0
HourlyWindSpeed             0
dtype: int64


## 2.5 Elimina vars Daily y renombra las Hourly

In [22]:
# 1. Elimina las columnas que empiezan con 'Daily'
columnas_daily = [col for col in df_expandido.columns if col.startswith('Daily')]
df_expandido.drop(columns=columnas_daily, inplace=True)

# 2. Revisa HourlyPresentWeatherType (condicion) y llena NaNs con 'CLR'
# Primero se renombra para trabajar directamente con el nombre final
df_expandido.rename(columns={'HourlyPresentWeatherType': 'condicion'}, inplace=True)
df_expandido['condicion'] = df_expandido['condicion'].fillna('CLR').replace('', 'CLR')

# 3. Separa DATE en FL_DATE (Y-M-D) y Hour (entero)
df_expandido['FL_DATE'] = df_expandido['DATE'].dt.date
df_expandido['Hour'] = df_expandido['DATE'].dt.hour.astype(int)

# 4. Cambia el nombre de DATE a FECHA_PARTIDA
df_expandido.rename(columns={'DATE': 'FECHA_PARTIDA'}, inplace=True)

# 5. Renombra el resto de las columnas Hourly
nuevos_nombres = {
    'HourlyDryBulbTemperature': 'temperatura',
    'HourlyRelativeHumidity': 'humedad',
    'HourlyStationPressure': 'presion',
    'HourlyVisibility': 'visibilidad',
    'HourlyWindSpeed': 'viento_velocidad'
}
df_expandido.rename(columns=nuevos_nombres, inplace=True)

# Verificación rápida del resultado
print("--- Columnas actuales en el DataFrame ---")
print(df_expandido.columns.tolist())
print("\n--- Ejemplo de las primeras filas ---")
print(df_expandido[['FECHA_PARTIDA', 'FL_DATE', 'Hour', 'condicion', 'temperatura']].head())

--- Columnas actuales en el DataFrame ---
['STATION', 'FECHA_PARTIDA', 'temperatura', 'humedad', 'presion', 'visibilidad', 'viento_velocidad', 'condicion', 'Month', 'FL_DATE', 'Hour']

--- Ejemplo de las primeras filas ---
        FECHA_PARTIDA     FL_DATE  Hour condicion  temperatura
0 2024-10-01 00:00:00  2024-10-01     0        RA        14.20
1 2024-10-01 00:53:00  2024-10-01     0        RA         7.20
2 2024-10-01 01:00:00  2024-10-01     1        RA         6.95
3 2024-10-01 01:53:00  2024-10-01     1        RA         6.70
4 2024-10-01 02:00:00  2024-10-01     2        RA         6.70


# 3. Preparación final df_procesado y df_expandido

## 3.1 df_expandido (clima)

In [23]:
df_procesado.columns

Index(['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DEL15', 'CRS_ARR_TIME',
       'ARR_TIME', 'ARR_DELAY', 'ARR_DEL15', 'CANCELLED', 'DIVERTED',
       'ACTUAL_ELAPSED_TIME', 'DISTANCE', 'WEATHER_DELAY',
       'HAS_FIRST_DEP_TIME', 'HAS_TOTAL_ADD_GTIME', 'FECHA_PARTIDA'],
      dtype='object')

In [24]:
airport_station_mapping.columns

Index(['IATA', 'STATION_ID', 'STATION_NAME', 'DIST_KM'], dtype='object')

### 3.1.1 Agrega col ORIGIN a df_expandido (clima)

In [25]:
# 1. Asegura que los tipos de datos coincidan para el cruce (ambos como string o ambos como int)
# Se usa string por seguridad ya que algunos IDs de estación tienen ceros a la izquierda
df_expandido['STATION'] = df_expandido['STATION'].astype(str)
airport_station_mapping['STATION_ID'] = airport_station_mapping['STATION_ID'].astype(str)

# 2. Crea un diccionario de mapeo para que sea más rápido
mapping_dict = dict(zip(airport_station_mapping['STATION_ID'], airport_station_mapping['IATA']))

# 3. Crea la columna ORIGIN en df_expandido
df_expandido['ORIGIN'] = df_expandido['STATION'].map(mapping_dict)

# 4. Verifica si algún STATION no encontró su IATA (esto es importante)
nulos_origin = df_expandido['ORIGIN'].isna().sum()
if nulos_origin > 0:
    print(f"Atención: {nulos_origin} filas no encontraron un aeropuerto IATA correspondiente.")
    print("Estaciones sin IATA:", df_expandido[df_expandido['ORIGIN'].isna()]['STATION'].unique())
else:
    print("Mapeo exitoso: Todas las estaciones tienen su código ORIGIN (IATA).")

Mapeo exitoso: Todas las estaciones tienen su código ORIGIN (IATA).


In [26]:
df_expandido.head(3)

,STATION,FECHA_PARTIDA,temperatura,humedad,presion,visibilidad,viento_velocidad,condicion,Month,FL_DATE,Hour,ORIGIN
0,3017,2024-10-01 00:00:00,14.20,34.0,840.20,16.09,3.50,RA,10,2024-10-01,0,DEN
1,3017,2024-10-01 00:53:00,7.20,44.0,845.60,16.09,2.10,RA,10,2024-10-01,0,DEN
2,3017,2024-10-01 01:00:00,6.95,44.5,845.45,16.09,2.35,RA,10,2024-10-01,1,DEN


## 3.2 df_procesado (vuelos)

In [27]:
df_procesado.head(3)

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,ARR_DELAY,ARR_DEL15,CANCELLED,DIVERTED,ACTUAL_ELAPSED_TIME,DISTANCE,WEATHER_DELAY,HAS_FIRST_DEP_TIME,HAS_TOTAL_ADD_GTIME,FECHA_PARTIDA
0,4,10,1,2,2024-10-01,9E,N132EV,5020,12953,LGA,...,52.0,1.0,0.0,0.0,135.0,431.0,0.0,0,0,2024-10-01T15:44:00
1,4,10,1,2,2024-10-01,9E,N132EV,5025,10431,AVL,...,72.0,1.0,0.0,0.0,111.0,599.0,33.0,0,0,2024-10-01T13:06:00
2,4,10,1,2,2024-10-01,9E,N132EV,5025,12953,LGA,...,43.0,1.0,0.0,0.0,128.0,599.0,43.0,0,0,2024-10-01T09:42:00


In [28]:
# 1. Asegura que FECHA_PARTIDA sea datetime
df_procesado['FECHA_PARTIDA'] = pd.to_datetime(df_procesado['FECHA_PARTIDA'])

# 2. Extrae la hora como entero
df_procesado['Hour'] = df_procesado['FECHA_PARTIDA'].dt.hour.astype(int)

# 3. Verifica que se haya creado correctamente
print("Primeras 5 filas con la nueva columna Hour:")
print(df_procesado[['FECHA_PARTIDA', 'Hour']].head())

Primeras 5 filas con la nueva columna Hour:
        FECHA_PARTIDA  Hour
0 2024-10-01 15:44:00    15
1 2024-10-01 13:06:00    13
2 2024-10-01 09:42:00     9
3 2024-10-01 21:06:00    21
4 2024-10-01 18:37:00    18


# 4. Combinación clima y procesado

## 4.1 Combina con FECHA PARTIDA exacta

In [29]:
# 1. Asegura que las columnas de unión tengan el mismo tipo (datetime)
df_procesado['FECHA_PARTIDA'] = pd.to_datetime(df_procesado['FECHA_PARTIDA'])
df_expandido['FECHA_PARTIDA'] = pd.to_datetime(df_expandido['FECHA_PARTIDA'])

# 2. Realiza el merge 'left' para conservar todos los vuelos de df_procesado
# Usa ORIGIN y FECHA_PARTIDA como llaves exactas
# Realiza el merge seleccionando las columnas de df_expandido "al vuelo"
df_procesado2 = pd.merge(
    df_procesado,
    df_expandido[['ORIGIN', 'FECHA_PARTIDA', 'temperatura', 'humedad', 'presion', 'visibilidad', 'viento_velocidad', 'condicion']],
    on=['ORIGIN', 'FECHA_PARTIDA'],
    how='left'
)

# 3. Verificación de resultados
match_exactos = df_procesado2['temperatura'].notna().sum()
total_vuelos = len(df_procesado2)

print(f"--- Resultado del cruce exacto ---")
print(f"Total de vuelos en df_procesado2: {total_vuelos}")
print(f"Vuelos con coincidencia exacta de clima: {match_exactos}")
print(f"Vuelos pendientes por rescatar (sin clima exacto): {total_vuelos - match_exactos}")

--- Resultado del cruce exacto ---
Total de vuelos en df_procesado2: 6915315
Vuelos con coincidencia exacta de clima: 251703
Vuelos pendientes por rescatar (sin clima exacto): 6663612


## 4.2 Combina con hora (entero)

In [30]:
# 1. Prepara la tabla de referencia con valores únicos (el primer registro de cada hora)
df_referencia = df_expandido.drop_duplicates(subset=['ORIGIN', 'Month', 'Hour'], keep='first')
df_referencia = df_referencia.rename(columns={'Month': 'MONTH'})

# 2. Columna de clima que se quiere rescatar
cols_clima = ['temperatura', 'humedad', 'presion', 'visibilidad', 'viento_velocidad', 'condicion']

# 3. Hace un merge temporal solo con las columnas necesarias
df_procesado2 = pd.merge(
    df_procesado2,
    df_referencia[['ORIGIN', 'MONTH', 'Hour'] + cols_clima],
    on=['ORIGIN', 'MONTH', 'Hour'],
    how='left',
    suffixes=('', '_REF')
)

# 4. Rellena los NaNs de las columnas originales usando las de REFERENCIA (_REF)
for col in cols_clima:
    df_procesado2[col] = df_procesado2[col].fillna(df_procesado2[f'{col}_REF'])
    # Elimina la columna de referencia inmediatamente para no dejar basura
    df_procesado2.drop(columns=[f'{col}_REF'], inplace=True)

# 5. Verificación final
print("--- Conteo de nulos corregido ---")
print(df_procesado2[cols_clima].isna().sum())

--- Conteo de nulos corregido ---
temperatura         876108
humedad             876108
presion             876108
visibilidad         876108
viento_velocidad    876108
condicion           876108
dtype: int64


# 5. Verificación de nulos y relación con 17 estaciones faltantes

In [31]:
stations_not_found = pd.read_csv(r'Base_climaNOAA\stations_not_found.csv')

In [32]:
# 1. Ver qué aeropuertos (ORIGIN) no tienen ni un solo dato de clima en df_procesado2
# Agrupa por ORIGIN para ver si el total de nulos en 'temperatura' es igual al total de filas
aeropuertos_sin_clima = df_procesado2.groupby('ORIGIN')['temperatura'].apply(lambda x: x.isnull().all())
lista_aeropuertos_vacios = aeropuertos_sin_clima[aeropuertos_sin_clima].index.tolist()

print(f"Aeropuertos en df_procesado2 que no tienen ningún dato de clima: {len(lista_aeropuertos_vacios)}")

# 2. Compara esos aeropuertos con df_expandido para obtener sus STATION_ID (STATION)
# Usam df_expandido como puente porque ahí sí tenemos la relación ORIGIN <-> STATION
estaciones_en_clima = df_expandido[df_expandido['ORIGIN'].isin(lista_aeropuertos_vacios)][['ORIGIN', 'STATION']].drop_duplicates()

# 3. Compara con stations_not_found['WBAN_ID']
# Asegura que ambos sean strings y sin espacios para evitar fallos por formato
estaciones_detectadas = set(estaciones_en_clima['STATION'].astype(str).str.strip())
estaciones_oficiales_faltantes = set(stations_not_found['WBAN_ID'].astype(str).str.strip())

# Identifica las estaciones que "sobran"
estaciones_infiltradas = estaciones_detectadas - estaciones_oficiales_faltantes

print("\n--- RESULTADO DEL ANÁLISIS ---")
print(f"Estaciones detectadas con nulos: {len(estaciones_detectadas)}")
print(f"Estaciones en lista oficial de faltantes: {len(estaciones_oficiales_faltantes)}")

if len(estaciones_infiltradas) > 0:
    print(f"\nLas {len(estaciones_infiltradas)} estaciones que DEBERÍAN tener datos pero están vacías son:")
    print(estaciones_infiltradas)
    
    # Ver a qué aeropuertos corresponden esas estaciones infiltradas
    mapping_infiltradas = estaciones_en_clima[estaciones_en_clima['STATION'].isin(estaciones_infiltradas)]
    print("\nAeropuertos correspondientes:")
    print(mapping_infiltradas)
else:
    print("\nNo se encontraron estaciones infiltradas. Todo coincide con la lista oficial.")

Aeropuertos en df_procesado2 que no tienen ningún dato de clima: 55

--- RESULTADO DEL ANÁLISIS ---
Estaciones detectadas con nulos: 0
Estaciones en lista oficial de faltantes: 17

No se encontraron estaciones infiltradas. Todo coincide con la lista oficial.


In [33]:
# 1. Identifica los 55 aeropuertos que no tienen clima
aeropuertos_vuelos = set(df_procesado2['ORIGIN'].unique())
aeropuertos_clima = set(df_expandido['ORIGIN'].unique())
huérfanos = list(aeropuertos_vuelos - aeropuertos_clima)

# 2. Busca sus estaciones en el mapeo
estaciones_huerfanas = airport_station_mapping[
    airport_station_mapping['IATA'].isin(huérfanos)
]['STATION_ID'].unique()

# 3. Compara con lista de 17 (stations_not_found)
oficiales_faltantes = set(stations_not_found['WBAN_ID'].astype(str).str.strip())
detectadas_huerfanas = set(pd.Series(estaciones_huerfanas).astype(str).str.strip())

sobrantes = detectadas_huerfanas - oficiales_faltantes
no_encontradas_en_vuelos = oficiales_faltantes - detectadas_huerfanas

print(f"Estaciones detectadas como ausentes: {len(detectadas_huerfanas)}")
print(f"Estaciones en lista oficial: {len(oficiales_faltantes)}")

if len(sobrantes) > 0:
    print(f"\nEstas {len(sobrantes)} estaciones NO están en tu lista de 17, pero sus aeropuertos no tienen clima:")
    print(sobrantes)

Estaciones detectadas como ausentes: 17
Estaciones en lista oficial: 17


# 6. Guarda df_procesado2

In [34]:
df_procesado2.columns

Index(['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DEL15', 'CRS_ARR_TIME',
       'ARR_TIME', 'ARR_DELAY', 'ARR_DEL15', 'CANCELLED', 'DIVERTED',
       'ACTUAL_ELAPSED_TIME', 'DISTANCE', 'WEATHER_DELAY',
       'HAS_FIRST_DEP_TIME', 'HAS_TOTAL_ADD_GTIME', 'FECHA_PARTIDA', 'Hour',
       'temperatura', 'humedad', 'presion', 'visibilidad', 'viento_velocidad',
       'condicion'],
      dtype='object')

In [35]:
# Guarda el DataFrame procesado
df_procesado2.to_csv('df_procesado2.csv', index=False, encoding='utf-8')

print("✅ Archivo 'df_procesado2.csv' guardado exitosamente.")
print(f"Dimensiones finales: {df_procesado2.shape[0]} filas y {df_procesado2.shape[1]} columnas.")

✅ Archivo 'df_procesado2.csv' guardado exitosamente.
Dimensiones finales: 6915315 filas y 35 columnas.
